This file will be used to contain data processing components

In [1]:
%pip install boto3 pyspark delta-spark python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pyspark.sql import SparkSession

In [3]:
# You can use the following to set the environment variables in the notebook if you don't set manually access key, secret key and endpoint in minio
os.environ['OBJ_STORAGE_ACCESS_KEY'] = 'minioadmin'
os.environ['OBJ_STORAGE_SECRET_KEY'] = 'minioadmin'
os.environ['OBJ_STORAGE_ENDPOINT'] = 'http://localhost:9000'

In [4]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [5]:
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [6]:
# You need to more configuration if you want to use minio as object storage 
# (hint: maybe you can using .config() method to set the configuration if you want using spark to read/write data from/to minio)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config('spark.hadoop.fs.s3a.path.style.access', "true") \
    .config("spark.jars.packages", "com.amazonaws:aws-java-sdk-bundle:1.12.690,org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", obj_storage_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key) \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", 'false') \
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", 15000) \
    .config("spark.hadoop.fs.s3a.connection.timeout", 15000) \
    .getOrCreate()

24/03/30 20:54:09 WARN Utils: Your hostname, Hoang-Nhans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/03/30 20:54:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/hoang-nhannguyen/.ivy2/cache
The jars for the packages stored in: /Users/hoang-nhannguyen/.ivy2/jars
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4bc1b091-9e9b-4b7e-85f8-ddc67ed5b94c;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/anaconda3/envs/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.amazonaws#aws-java-sdk-bundle;1.12.690 in central
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.5.2 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.17 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found

In [7]:
st_df = spark.read.format("json").option("multiLine", True).option("header",True).option("inferschema",True).load(path_1)
nd_df = spark.read.format("json").option("multiLine", True).option("header",True).option("inferschema",True).load(path_2)
rd_df = spark.read.format("json").option("multiLine", True).option("header",True).option("inferschema",True).load(path_3)
schema = st_df.schema

24/03/30 20:54:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [8]:
json_path = "s3a://data/data-result/result.json"
merged_df = st_df.union(nd_df).union(rd_df)
merged_df.write.format("json").mode("overwrite").save(json_path)

24/03/30 20:54:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
24/03/30 20:54:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
24/03/30 20:54:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
24/03/30 20:54:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [9]:
df = spark.read.json(json_path)

In [10]:
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0008| Strawberry Frosted|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5.

In [11]:
df.count()

19

In [12]:
df = df.distinct()

In [13]:
df.count()

18

In [14]:
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0011|  Devil's Food Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5.

In [15]:
from pyspark.sql.functions import col

nested_df = df.select(
    col("id"),
    col("type"),
    col("name"),
    col("ppu"),
    col("batters").cast("string").alias("batters"),
    col("topping").cast("string").alias("topping"),
    col("filling").cast("string").alias("filling")
)

In [16]:
nested_df.show()

+----+-----+-------------------+----+--------------------+--------------------+--------------------+
|  id| type|               name| ppu|             batters|             topping|             filling|
+----+-----+-------------------+----+--------------------+--------------------+--------------------+
|0004|donut|              Jelly|0.65|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0011|donut|  Devil's Food Cake|0.75|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0010|donut|     Blueberry Cake|0.55|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0007|donut|    Vanilla Frosted|0.75|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0006|donut|     Cinnamon Twist|0.85|{[{1001, Regular}...|[{5001, None}, {5...|[{6001, None}, {6...|
|0009|donut|     Chocolate Cake|0.75|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0005|donut|     Custard-Filled|0.75|{[{1001, Regular}...|[{5001, None}, {5...|            

In [17]:
csv_path = "s3a://data/data-result/result.csv"

nested_df.write.mode("overwrite").csv(csv_path, header=True)

24/03/30 20:54:18 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
24/03/30 20:54:18 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [18]:
df = spark.read.csv(csv_path, header=True)

In [19]:
df.show()

+----+-----+-------------------+----+--------------------+--------------------+--------------------+
|  id| type|               name| ppu|             batters|             topping|             filling|
+----+-----+-------------------+----+--------------------+--------------------+--------------------+
|0004|donut|              Jelly|0.65|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0011|donut|  Devil's Food Cake|0.75|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0010|donut|     Blueberry Cake|0.55|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0007|donut|    Vanilla Frosted|0.75|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0006|donut|     Cinnamon Twist|0.85|{[{1001, Regular}...|[{5001, None}, {5...|[{6001, None}, {6...|
|0009|donut|     Chocolate Cake|0.75|{[{1001, Regular}...|[{5001, None}, {5...|                NULL|
|0005|donut|     Custard-Filled|0.75|{[{1001, Regular}...|[{5001, None}, {5...|            

24/03/30 20:54:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/03/30 21:12:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 420943 ms exceeds timeout 120000 ms
24/03/30 21:12:40 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/30 21:12:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.Rpc